In [ ]:
using BitemporalPostgres
using LifeInsuranceDataModel, LifeInsuranceProduct
using SearchLight
using SearchLightPostgreSQL
using TimeZones
SearchLight.Configuration.load() |> SearchLight.connect
cid = 1
h = find(Contract, SQLWhereExpression("id =?", cid))[1].ref_history
vi = find(ValidityInterval, SQLWhereExpression("ref_history=?", h), order=["ValidityInterval.id"])[1];
txntime = vi.tsdb_validfrom
reftime = vi.tsworld_validfrom

cs = csection(cid, txntime, reftime)

ti = cs.product_items[1].tariff_items[1]

calculate!(ti)

println(ti.tariff_ref.rev)

In [13]:
using Dates, LifeContingencies
using MortalityTables
using Yields
import LifeContingencies: V, ä     # pull the shortform notation into scope

# load mortality rates from MortalityTables.jl
vbt2001 = MortalityTables.table("2001 VBT Residual Standard Select and Ultimate - Male Nonsmoker, ANB")

issue_age = 30

life = SingleLife(                 # The life underlying the risk
    mortality=vbt2001.select[issue_age],    # -- Mortality rates
)

yield = Yields.Constant(0.0125)      # Using a flat 1,25% interest rate

lc = LifeContingency(life, yield)  # LifeContingency joins the risk with interest


r1 = ä(life, yield)
r2 = ä(life, yield, 1)
r3 = ä(life, yield, 2)
r4 = ä(life, yield, 30)

println("r1")
println(r1)
println("r2")
println(r2)
println("r3")
println(r3)
println("r4")
println(r4)

r1
36.61266492009452
r2
1.0
r3
1.9873283950617284
r4
24.718427041373936
